In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily

from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from shapely.geometry import LineString
from shapely.ops import split

import matplotlib.pyplot as plt
import matplotlib.colors as colors

# nossos arquivos
from utils import *
from satelites import *
from graphic import *

from pyproj import Geod
from typing import Callable

In [28]:
class ExploreIA:

    def __init__(self, data: pd.DataFrame, quadrat_width: float = 0.005):
        self.data = data
        self.satellites_data = SatellitesMeasureGeometry(data)
        self.geod = Geod(ellps="WGS84")
        self.dataframe = self.satellites_data.get_satelites_measures_area()
        self.quadrat_width = quadrat_width

    def prepare(self) -> gpd.GeoDataFrame:
        quads_df = grid_gdf(self.dataframe, quadrat_width=self.quadrat_width)
        join_dataframe = gpd.sjoin(self.dataframe, quads_df, predicate="intersects")

        d = { }
        for satelite in self.data['satelite'].unique():
            d[satelite] = np.zeros(len(quads_df))
        for index in join_dataframe['index_right'].unique():
            polygon = quads_df.iloc[index].geometry
            precise_matches: gpd.GeoDataFrame = join_dataframe[join_dataframe['index_right'] == index]
            for satelite in precise_matches['satelite'].unique():
                values = precise_matches[precise_matches['satelite'] == satelite]
                areas = np.array([polygon.intersection(x).area for x in values.geometry])
                d[satelite][index] = areas.sum() / polygon.area

        return gpd.GeoDataFrame(d, geometry=quads_df.geometry, crs=quads_df.crs)


In [6]:
df = read_burn_df()

In [8]:
path, row, quadrat_width = 221, 67, 0.005
temp = sub_space_by_landsat(df.query("'2021-06-18 00:00:00-03:00' < datahora < '2021-07-03 23:59:59-03:00'"), path, row)
temp

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp,simp_satelite,sensor
75911,2021-06-18 09:44:19-03:00,GOES-16,BRASIL,TOCANTINS,PONTE ALTA DO TOCANTINS,CERRADO,31,0.0,True,-10.810000,-47.010000,NaN,GOES-16,ABI
75912,2021-06-18 09:44:19-03:00,GOES-16,BRASIL,TOCANTINS,PONTE ALTA DO TOCANTINS,CERRADO,28,0.0,True,-10.790000,-47.010000,NaN,GOES-16,ABI
75935,2021-06-18 09:54:18-03:00,GOES-16,BRASIL,TOCANTINS,PONTE ALTA DO TOCANTINS,CERRADO,28,0.0,True,-10.790000,-47.010000,NaN,GOES-16,ABI
75936,2021-06-18 09:54:18-03:00,GOES-16,BRASIL,TOCANTINS,PONTE ALTA DO TOCANTINS,CERRADO,31,0.0,True,-10.810000,-47.010000,NaN,GOES-16,ABI
75973,2021-06-18 10:04:18-03:00,GOES-16,BRASIL,TOCANTINS,PONTE ALTA DO TOCANTINS,CERRADO,28,0.0,True,-10.790000,-47.010000,NaN,GOES-16,ABI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17297,2021-07-03 14:24:22-03:00,GOES-16,BRASIL,TOCANTINS,MATEIROS,CERRADO,46,0.0,True,-10.270000,-46.720000,NaN,GOES-16,ABI
17325,2021-07-03 14:44:22-03:00,GOES-16,BRASIL,TOCANTINS,MATEIROS,CERRADO,49,0.0,True,-10.270000,-46.690000,NaN,GOES-16,ABI
17326,2021-07-03 14:44:22-03:00,GOES-16,BRASIL,TOCANTINS,MATEIROS,CERRADO,46,0.0,True,-10.270000,-46.720000,NaN,GOES-16,ABI
17787,2021-07-03 15:14:22-03:00,GOES-16,BRASIL,TOCANTINS,MATEIROS,CERRADO,23,0.0,True,-10.320000,-47.020000,NaN,GOES-16,ABI


In [38]:
geometry = get_landsat_geometry(path, row)
reference = normalize_gdf(gpd.read_file('tiff/LS8_AQM_221_067_20210703_0107'), geometry, quadrat_width)
reference

,value,geometry
0,0.0,"POLYGON ((-47.68000 -10.76500, -47.68000 -10.7..."
1,0.0,"POLYGON ((-47.68000 -10.76000, -47.68000 -10.7..."
2,0.0,"POLYGON ((-47.68000 -10.75500, -47.68000 -10.7..."
3,0.0,"POLYGON ((-47.68000 -10.75000, -47.68000 -10.7..."
4,0.0,"POLYGON ((-47.68000 -10.74500, -47.68000 -10.7..."
...,...,...
105532,0.0,"POLYGON ((-45.68500 -9.50000, -45.68500 -9.495..."
105533,0.0,"POLYGON ((-45.68500 -9.49500, -45.68500 -9.490..."
105534,0.0,"POLYGON ((-45.68500 -9.49000, -45.68500 -9.485..."
105535,0.0,"POLYGON ((-45.68500 -9.48500, -45.68500 -9.480..."


In [29]:
data = ExploreIA(temp).prepare()
data

,GOES-16,TERRA_M-T,MSG-03,NOAA-20,AQUA_M-T,NPP-375,NOAA-19,NPP-375D,TERRA_M-M,AQUA_M-M,geometry
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-47.51500 -10.88500, -47.51500 -10.8..."
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-47.51500 -10.88000, -47.51500 -10.8..."
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-47.51500 -10.87500, -47.51500 -10.8..."
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-47.51500 -10.87000, -47.51500 -10.8..."
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-47.51500 -10.86500, -47.51500 -10.8..."
...,...,...,...,...,...,...,...,...,...,...,...
109797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-45.81500 -9.30000, -45.81500 -9.295..."
109798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-45.81500 -9.29500, -45.81500 -9.290..."
109799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-45.81500 -9.29000, -45.81500 -9.285..."
109800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-45.81500 -9.28500, -45.81500 -9.280..."


In [39]:
original_geometry = data['geometry']
data['geometry'] = data.representative_point()
join_gpd = gpd.sjoin(reference, data, predicate="contains", lsuffix='reference', rsuffix='other')
data['geometry'] = original_geometry


In [40]:
join_gpd

,value,geometry,index_other,GOES-16,TERRA_M-T,MSG-03,NOAA-20,AQUA_M-T,NPP-375,NOAA-19,NPP-375D,TERRA_M-M,AQUA_M-M
2665,0.0,"POLYGON ((-47.51500 -10.79000, -47.51500 -10.7...",19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2666,0.0,"POLYGON ((-47.51500 -10.78500, -47.51500 -10.7...",20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2667,0.0,"POLYGON ((-47.51500 -10.78000, -47.51500 -10.7...",21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2668,0.0,"POLYGON ((-47.51500 -10.77500, -47.51500 -10.7...",22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2669,0.0,"POLYGON ((-47.51500 -10.77000, -47.51500 -10.7...",23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103816,0.0,"POLYGON ((-45.81500 -9.48500, -45.81500 -9.480...",109760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103817,0.0,"POLYGON ((-45.81500 -9.48000, -45.81500 -9.475...",109761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103818,0.0,"POLYGON ((-45.81500 -9.47500, -45.81500 -9.470...",109762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103819,0.0,"POLYGON ((-45.81500 -9.47000, -45.81500 -9.465...",109763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
variable_names.tolist()

['GOES-16',
 'TERRA_M-T',
 'MSG-03',
 'NOAA-20',
 'AQUA_M-T',
 'NPP-375',
 'NOAA-19',
 'NPP-375D',
 'TERRA_M-M',
 'AQUA_M-M']

In [54]:
from pysal.model import spreg

variable_names = data.columns.values[0:-1].tolist()
m1 = spreg.OLS(
    join_gpd[["value"]].values,
    join_gpd[variable_names].values,
    name_y="burn_factor",
    name_x=variable_names
)

In [59]:
print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  : burn_factor                Number of Observations:       98812
Mean dependent var  :      0.0187                Number of Variables   :          11
S.D. dependent var  :      0.1051                Degrees of Freedom    :       98801
R-squared           :      0.2059
Adjusted R-squared  :      0.2058
Sum squared residual:     866.055                F-statistic           :   2561.0286
Sigma-square        :       0.009                Prob(F-statistic)     :           0
S.E. of regression  :       0.094                Log likelihood        :   93829.322
Sigma-square ML     :       0.009                Akaike info criterion : -187636.644
S.E of regression ML:      0.0936                Schwarz criterion     : -187532.133

-----------------------------------------------------------------------------

In [65]:
join_gpd[reference['value'] > 0.5]

/Users/josebraz/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


,value,geometry,index_other,GOES-16,TERRA_M-T,MSG-03,NOAA-20,AQUA_M-T,NPP-375,NOAA-19,NPP-375D,TERRA_M-M,AQUA_M-M
2888,0.821259,"POLYGON ((-47.51000 -10.48000, -47.51000 -10.4...",403,0.000000,0.000000,0.0,1.792821,0.461140,0.853257,0.0,0.0,0.0,0.0
2889,0.804482,"POLYGON ((-47.51000 -10.47500, -47.51000 -10.4...",404,0.000000,0.000000,0.0,0.254377,0.085491,0.006433,0.0,0.0,0.0,0.0
3054,0.951153,"POLYGON ((-47.50500 -10.47500, -47.50500 -10.4...",726,0.000000,0.000000,0.0,0.843278,0.269922,0.000000,0.0,0.0,0.0,0.0
3056,0.534759,"POLYGON ((-47.50500 -10.46500, -47.50500 -10.4...",728,0.000000,0.000000,0.0,0.000000,0.000000,0.478721,0.0,0.0,0.0,0.0
3607,0.637504,"POLYGON ((-47.49000 -10.33500, -47.49000 -10.3...",1720,0.000000,0.000000,0.0,0.381153,0.000000,1.252083,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93031,0.534985,"POLYGON ((-46.04000 -10.56500, -46.04000 -10.5...",95054,18.000000,0.000000,0.0,0.000000,0.000000,2.730497,0.0,0.0,0.0,0.0
93032,0.635154,"POLYGON ((-46.04000 -10.56000, -46.04000 -10.5...",95055,18.000000,0.000000,0.0,0.000000,0.000000,3.825104,0.0,0.0,0.0,0.0
93033,0.689459,"POLYGON ((-46.04000 -10.55500, -46.04000 -10.5...",95056,16.762709,0.000000,0.0,0.000000,0.000000,0.241778,0.0,0.0,0.0,0.0
93976,0.780005,"POLYGON ((-46.02500 -10.56500, -46.02500 -10.5...",96020,18.000000,0.119265,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [83]:
teste = join_gpd.loc[93976][variable_names].values
m1.betas[0] + (m1.betas[1:][:, 0] * teste).sum()

array([0.64582974])